**Call이 어디서 발생하느냐에 따라 user Call과 Recursive Call로 구분.**

# **포맷팅**

- SQL 트레이스 파일을 TKProf 유틸리티로 포맷팅하면 OVERALL Total 통계가 나옴.
- NON-RECURSIVE 통계가 User Call에 해당.
- 아래쪽 RECURSIVE 통계가 Recursive Call에 해당.



```sql
================================================================================
OVERALL TOTALS FOR ALL NON-RECURSIVE STATEMENTS

call     count       cpu    elapsed       disk      query    current        rows
------- ------  -------- ---------- ---------- ---------- ----------  ----------
Parse        4      0.02       0.03          0        121          0           0
Execute      4      0.03       0.03          4         95       2915           0
Fetch       30      0.04       0.02          0        122          0        2859
------- ------  -------- ---------- ---------- ---------- ----------  ----------
total       38      0.09       0.09          4        338       2915        2859

Misses in library cache during parse: 2
================================================================================
================================================================================
 OVERALL TOTALS FOR ALL RECURSIVE STATEMENTS

 call     count       cpu    elapsed       disk      query    current        rows
 ------- ------  -------- ---------- ---------- ---------- ----------  ----------
 Parse        4      0.00       0.00          0          0          0           0
 Execute     53      0.00       0.00          0          0          0           0
 Fetch       53      0.00       0.00         80        277          0           3
 ------- ------  -------- ---------- ---------- ---------- ----------  ----------
 total      110      0.00       0.00         80        277          0           3
 ================================================================================
 ```


# **user Call**

- OCI(Oracle Call Interface)를 통해 오라클 외부로부터 들어오는 Call.
- Peak 시간대에 시스템 장애를 발생시키는 가장 큰 주범은 User Call.
- User Call이 많이 발생되도록 개발된 애플리케이션은 결코 좋은 성능을 낼 수 없음.
- DBMS 성능과 확장성(Scalability)를 높이려면 User Call을 최소화 하려는 노력이 무엇보다 중요.

**User Call 최소화를 위한 노력**

- User Call 최소화를 위해 아래와 같은 기능과 기술을 적극적으로 활용.
  - Loop 쿼리를 해소하고 집합적 사고를 통해 One-SQL로 구현
  - Array 단위로 Fetch하는 Array Processing을 활용.
    - 또는 Bulk Insert/Update/Delete 가 있음.
  - 부분범위처리 원리를 활용.
  - 효과적인 화면 페이지 처리를 구현.

# **Recursive Call**
- 오라클 내부에서 발생하는 Call.
- Recursive Call의 종류
  - SQL 파싱과 최적화 과정에서 발생하는 Data Dictionary 조회
  - PL/SQL로 작성된 사용자 정의 함수/프로시저/트리거 내에서의 SQL 수행 등

**Recursive Call 최소화를 위한 노력**

- 바인드변수를 적극적으로 사용해 하드파싱 횟수를 줄임.
- PL/SQL로 작성한 프로그램을 이해하고 시기 적절하게 사용.



```sql
[PL/SQL 예시]
================================================================================
call     count       cpu    elapsed       disk      query    current        rows
------- ------  -------- ---------- ---------- ---------- ----------  ----------
Parse        0      0.00       0.00          0          0          0           0
Execute    493      0.01       0.00          0          0          0           0
Fetch      493      0.03       0.02          0       3451          0         493
------- ------  -------- ---------- ---------- ---------- ----------  ----------
total      986      0.04       0.02          0       3451          0         493

Misses in library cache during parse: 0
Optimizer mode: ALL_ROWS
Parsing user id: 41     (recursive depth: 1)
================================================================================
```

- recursive depth는 PL/SQL실행시에 나오는 결과에서 프로시저의 호출 횟수를 나타냄.
- recursive depth가 2이상이면 특정 프로시저에서 또 다른 프로시져를 호출한 경우
- 이 때 트레이스 결과는 마지막 프로시저에서 사용된 SQL에 대한 수행 결과를 보임.
- PL/SQL은 가상머신(Virtual Machine)상에서 수행되는 인터프리터(Interpreter)언어이므로 빈번한 호출 시 컨텍스트 스위칭(Context Switching)때문에 성능이 매우 나빠짐.
- 성능을 위해서라면 PL/SQL에 대한 지나친 모듈화는 지양.
- 대용량 데이터 조회시에 함수를 잘못 사용하면 건건이 함수 호출이 발생되어 성능이 극도로 제한될 수 있는 등의 문제가 생김.
  - 조인 또는 스칼라 서브쿼리 형태로 변환하려는 노력이 필요.
